In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin

In [3]:
url = "https://bankofjordan.com/"
base_url = "https://bankofjordan.com"
headers = {"User-Agent": "Mozilla/5.0"}


In [5]:
folder_name = "Bank_of_Jordan_Images"
os.makedirs(folder_name, exist_ok=True)

response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")


In [7]:
data = []

# 1. السحب من الهوم بيج سلايدر والصورة الرئيسية
main_slider = soup.select_one(".homepage_banner .home_business_banner img")
if main_slider:
    img_src = main_slider.get("src", "").strip()
    if img_src:
        full_img_url = urljoin(base_url, img_src)
        image_name = img_src.split("/")[-1].split("?")[0]
        
        # نحاول نجيب الرابط المرتبط بالصورة (لو موجود)
        a_tag = soup.select_one(".home_business_banner_content a")
        article_link = urljoin(base_url, a_tag["href"]) if a_tag else url

        # تحميل الصورة
        img_path = os.path.join(folder_name, image_name)
        img_resp = requests.get(full_img_url)
        with open(img_path, "wb") as f:
            f.write(img_resp.content)

        data.append({
            "Bank_Name": "Bank of Jordan",
            "Image_Name": image_name,
            "Image_Link": full_img_url,
            "Article_Link": article_link
        })
        print(f"✔️ تم تحميل صورة السلايدر الرئيسي: {image_name}")

# 2. سحب الصور من كروت الخدمات
program_cards = soup.select(".program_boxes .image-box a")
for card in program_cards:
    img_tag = card.find("img")
    if img_tag:
        img_src = img_tag.get("src", "").strip()
        full_img_url = urljoin(base_url, img_src)
        image_name = img_src.split("/")[-1].split("?")[0]
        article_link = urljoin(base_url, card["href"])

        # تحميل الصورة
        img_path = os.path.join(folder_name, image_name)
        img_resp = requests.get(full_img_url)
        with open(img_path, "wb") as f:
            f.write(img_resp.content)

        data.append({
            "Bank_Name": "Bank of Jordan",
            "Image_Name": image_name,
            "Image_Link": full_img_url,
            "Article_Link": article_link
        })
        print(f"✔️ تم تحميل صورة كارد: {image_name}")

# 3. سحب صورة التطبيق (download bank application section)
download_app_img = soup.select_one(".download_bankapplication img")
if download_app_img:
    img_src = download_app_img.get("src", "").strip()
    full_img_url = urljoin(base_url, img_src)
    image_name = img_src.split("/")[-1].split("?")[0]

    # نحاول نربطها برابط التطبيق
    parent_a = download_app_img.find_parent("a")
    article_link = urljoin(base_url, parent_a["href"]) if parent_a else url

    # تحميل الصورة
    img_path = os.path.join(folder_name, image_name)
    img_resp = requests.get(full_img_url)
    with open(img_path, "wb") as f:
        f.write(img_resp.content)

    data.append({
        "Bank_Name": "Bank of Jordan",
        "Image_Name": image_name,
        "Image_Link": full_img_url,
        "Article_Link": article_link
    })
    print(f"✔️ تم تحميل صورة التطبيق: {image_name}")


✔️ تم تحميل صورة السلايدر الرئيسي: ea36b94e-0b85-60d31ba99f36-1918x1100.jpg
✔️ تم تحميل صورة كارد: 7e8f937e-49dd-5f841799851c-585x274.jpg
✔️ تم تحميل صورة كارد: 8722d884-5678-5f61dca25781-585x274.jpg
✔️ تم تحميل صورة كارد: 1a0831fe-19ae-5f61e222dc72-585x274.jpg
✔️ تم تحميل صورة كارد: 1a5804c4-8ffe-65b221da081b-585x274.jpg
✔️ تم تحميل صورة كارد: d3e18aba-65f0-5f61e3186658-585x274.jpg
✔️ تم تحميل صورة كارد: 60f8a475-2cb9-5f61e213b8f8-585x274.jpg
✔️ تم تحميل صورة كارد: 26fb4905-d816-5f1fd58951aa-585x274.jpg
✔️ تم تحميل صورة كارد: 1f0b419c-8f00-6475c2e8e5ff-775x275.jpg
✔️ تم تحميل صورة التطبيق: a7d5df3f-08b0-5f09f7e96031-215x65.png


In [11]:
# news 
news_links = soup.select(".aqsati_program_banneright_content h6 a")
for news in news_links:
    if news.get("href"):
        article_link = urljoin(base_url, news["href"])

        data.append({
            "Bank_Name": "Bank of Jordan",
            "Image_Name": "No_Image",
            "Image_Link": "No_Image_Link",
            "Article_Link": article_link
        })
        print(f"📰 تم إضافة رابط خبر: {article_link}")


📰 تم إضافة رابط خبر: https://bankofjordan.com/ar/news/announcement-of-the-distribution-of-dividends-for-bank-of-jordan-shares-2
📰 تم إضافة رابط خبر: https://bankofjordan.com/ar/news/announcement-to-the-shareholders-of-bank-of-jordan-plc-to-attend-the-ordinary-general-assembly-meeting-2


In [13]:
df = pd.DataFrame(data)
df

,Bank_Name,Image_Name,Image_Link,Article_Link
0,Bank of Jordan,ea36b94e-0b85-60d31ba99f36-1918x1100.jpg,https://bankofjordan.com/uploads/2021/06/ea36b...,https://bankofjordan.com/ar/digital-banking/do...
1,Bank of Jordan,7e8f937e-49dd-5f841799851c-585x274.jpg,https://bankofjordan.com/uploads/2020/10/7e8f9...,https://bankofjordan.com/ar/cards
2,Bank of Jordan,8722d884-5678-5f61dca25781-585x274.jpg,https://bankofjordan.com/uploads/2020/09/8722d...,https://bankofjordan.com/ar/accounts
3,Bank of Jordan,1a0831fe-19ae-5f61e222dc72-585x274.jpg,https://bankofjordan.com/uploads/2020/09/1a083...,https://bankofjordan.com/ar/loans/housing-2
4,Bank of Jordan,1a5804c4-8ffe-65b221da081b-585x274.jpg,https://bankofjordan.com/uploads/2024/01/1a580...,https://www.dic.gov.jo/Default/Ar
5,Bank of Jordan,d3e18aba-65f0-5f61e3186658-585x274.jpg,https://bankofjordan.com/uploads/2020/09/d3e18...,https://bankofjordan.com/ar/loans/personal
6,Bank of Jordan,60f8a475-2cb9-5f61e213b8f8-585x274.jpg,https://bankofjordan.com/uploads/2020/09/60f8a...,https://bankofjordan.com/ar/loans/auto-loans
7,Bank of Jordan,26fb4905-d816-5f1fd58951aa-585x274.jpg,https://bankofjordan.com/uploads/2020/07/26fb4...,https://bankofjordan.com/ar/programs/sanabel
8,Bank of Jordan,1f0b419c-8f00-6475c2e8e5ff-775x275.jpg,https://bankofjordan.com/uploads/2023/05/1f0b4...,https://bankofjordan.com/ar/business-products/...
9,Bank of Jordan,a7d5df3f-08b0-5f09f7e96031-215x65.png,https://bankofjordan.com/uploads/2020/07/a7d5d...,https://play.google.com/store/apps/details?id=...
